In [1]:
import pandas as pd
import numpy as np
from itertools import combinations 
import pickle as pickle
import random

In [2]:
d_advice = pd.read_csv('~/data/final_proj_data_preprocessed.csv')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,8,11,13,15,16,17,18,19,21,22,30,38,41,42,44,47,48,50,51,74,75,87,89,94,117,119,123) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
d_advice.shape

(39440, 136)

In [26]:
n = d_advice.shape[0]

In [ ]:
sub_list = ['depression', 'relationship_advice', 'offmychest',
'IAmA', 'needadvice', 'tifu', 'confessions','anxiety', 'confession', 'socialanxiety','socialskills','trueoffmychest']

In [28]:
for sub in sub_list:
    d_sub = pd.read_csv('expanded_%s_preprocessed.csv'%sub)
    n = n + d_sub.shape[0]

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,5,9,10,14,15,118,119,120,121,124,146,147,148,149,152,167,168,169,170,173,205,264,265,266,267,270,292,293,294,295,298,299,300,301,302,305,370,371,372,373,376,385,386,387,388,391,406,407,408,409,412,435,436,437,438,439,442,458,459,460,461,464,486,487,488,489,492,500,501,502,503,506,507,508,509,510,513,570,571,572,573,576,577,578,579,580,583,597,599,601,602,603,604,646) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
n

226403

In [44]:
d = d_advice[['id', 'author', 'title', 'selftext']]

In [45]:
authors = d.groupby('author').id.count().to_frame()
same_authors = authors[authors.id>1]
same_authors_d = d[d.author.isin(same_authors.index)]
text_dict = same_authors_d.groupby('author')['selftext'].apply(list).to_dict()

for author, t in text_dict.items():
    #flat_list = [item for sublist in t for item in sublist]
    for c in combinations(t,2):
        train = train.append({"seq_A":c[0], "seq_B":c[1], 'label':1,'author':author}, ignore_index=True)


for row in train.iterrows(): 
    current_author = row[1]['author']
    d_pool = d[d.author!=current_author]
    s_text = random.choice(d_pool.selftext.to_list())
    train = train.append({"seq_A":row[1]['seq_A'], "seq_B":s_text,
                                      'label':0,'author':'paired_authors'}, ignore_index=True)


In [ ]:
for sub in sub_list:
    d_sub = pd.read_csv('expanded_%s_preprocessed.csv'%sub)
    d = d_sub[['id', 'author', 'title', 'selftext']]
    
    authors = d.groupby('author').id.count().to_frame()
    same_authors = authors[authors.id>1]
    same_authors_d = d[d.author.isin(same_authors.index)]
    text_dict = same_authors_d.groupby('author')['selftext'].apply(list).to_dict()
    
    column_names = ["seq_A", "seq_B", "label",'author']
    train_partial = pd.DataFrame(columns = column_names)

    for author, t in text_dict.items():
        #flat_list = [item for sublist in t for item in sublist]
        for c in combinations(t,2):
            train_partial = train_partial.append({"seq_A":c[0], "seq_B":c[1], 'label':1,'author':author}, ignore_index=True)


    for row in train_partial.iterrows(): 
        current_author = row[1]['author']
        d_pool = d[d.author!=current_author]
        s_text = random.choice(d_pool.selftext.to_list())
        train_partial = train_partial.append({"seq_A":row[1]['seq_A'], "seq_B":s_text,
                                      'label':0,'author':'paired_authors'}, ignore_index=True)
    
    train = train.append(train_partial)

In [68]:
train.to_csv('bert_finetuning_train.pickle.csv')